In [1]:
from qiskit import QuantumCircuit, Aer, assemble
from numpy.random import randint
import numpy as np

In [2]:
def encode_message(bits, bases):
    message = []
    # Iterate through each bit to encode
    for i in range(n):
        qc = QuantumCircuit(1,1)
        # Apply gates as given in row #5 in table 7.4.
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                # Do nothing for the bit value 0 in Z-basis (|0⟩ state)
                pass 
            else:
                # Apply X gate to flip the qubit if bit value is 1
                qc.x(0) 
        else: # Prepare qubit in X-basis
            if bits[i] == 0:
                # Apply Hadamard gate to prepare |+⟩ state for bit value 0
                qc.h(0)
            else:
                # Apply X gate to prepare |1⟩ state first
                qc.x(0)
                # Then apply Hadamard gate to get |1⟩ in X-basis as |-⟩        
                # state
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

In [3]:
def measure_message(message, bases):
    backend = Aer.get_backend('aer_simulator')
    measurements = []
    # Iterate through all the qubits
    for q in range(n):
        # If basis is Z, then apply Identity gate, or do nothing, just 
        # measure
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        # If basis is X, apply Hadamard gate and then measure
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
        # Get the Aer simulator backend for running quantum circuits
        aer_sim = Aer.get_backend('aer_simulator')
        # Assemble the quantum object (qobj) from the encoded message
        qobj = assemble(message[q], shots=1, memory=True)
        # Run the quantum object on the simulator and obtain the result
        result = aer_sim.run(qobj).result()
        # Retrieve the measured bit from the result's memory
        # The first element in the memory is converted to an integer 	
        measured_bit = int(result.get_memory()[0])
        # Append the measured bit to the measurements list for further use
        measurements.append(measured_bit)
    return measurements, message

In [4]:
def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
    return good_bits

In [5]:
def sample_bits(bits, selection):
    sample = []
    for i in selection:
        # use np.mod to make sure the bit we sample is always in the list 
        # range
        i = np.mod(i, len(bits))
        # pop(i) removes the element of the
        # list at index 'i'
        sample.append(bits.pop(i))
    return sample

In [6]:
def print_results(alice_key, bob_key, bob_sample, alice_sample):
    #print('Alice sample ', alice_sample)
    #print('Bob sample ', bob_sample)
    print('Alice key ', alice_key)
    print('Bob key ', bob_key)
    print('Key matches? ', alice_key == bob_key)
    print('Sample matches? ',alice_sample == bob_sample)

In [7]:
def validate(alice_key, bob_key, bob_sample, alice_sample ):
    global detected, undetected
    print_results(alice_key, bob_key, bob_sample, alice_sample)
    # No eavesdropping
    if alice_key == bob_key:
        if bob_sample == alice_sample:
            print('There was no eavesdropper')
        else:
            print("Strange error!")
    else:
        # Eve eavesdropped
        if bob_sample == alice_sample:
            print("Eve went undetected!")
            undetected += 1
        else:
            print("Eve's interference was detected.")
            detected += 1

In [14]:
def main(n, sample_size, interception=True):
    # Step 1
    # Generate 'n' random bits (0s and 1s) for Alice's message
    alice_bits = randint(2, size=n)
    # Generate 'n' random bases (0s and 1s) to determine measurement basis
    alice_bases = randint(2, size=n)
    #alice_bits=[0,1,0,1]
    #alice_bases=[0,0,1,1]
    # Step 2 
    # Encode Alice's bits into quantum messages based on her chosen bases
    message = encode_message(alice_bits, alice_bases)
    
    if interception:
        # Interception!
        # Generate 'n' random bases for Eve's measurements
        eve_bases = randint(2, size=n)
        intercepted_message, eve_mesg = measure_message(message, eve_bases)
        #eve_bases = [1,1,0,0]
    else:
        eve_mesg = None
        
    # Step 3
    # Generate 'n' random bases to measure the incoming message by Bob
    bob_bases = randint(2, size=n)
    # Alternative option where Bob can use Alice's bases (commented out)
    #bob_bases = alice_bases
    # Measure the quantum message using Bob's bases and retrieve results
    bob_results, bob_mesg = measure_message(message, bob_bases)

    
    # Step 4
        # Alice and Bob will discard qubits where their bases do not match
    bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
    alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
    
    # Step 5
    # Select bit positions to determine sample bits
    bit_selection = randint(n, size=sample_size)
    # Bob will select sample bits to send to Alice
    bob_sample = sample_bits(bob_key, bit_selection)
    # Alice will select sample bits to send to Bob
    alice_sample = sample_bits(alice_key, bit_selection)
    
        # Verify whether Eve’s interception was detected or not
    
    validate(alice_key, bob_key, bob_sample, alice_sample )
    
    # Returns eve’s and bob’s messages
    return eve_mesg, bob_mesg

In [29]:
# 100 qubits will be randomly generated along with the bases.
n = 100
# Sample size of qubits that will be shared by Alice and Bob
sample_size = 10
# Initialize counts.
detected, undetected = 0, 0
# Run the main function 10 times.
for i in range(10):
    print(f'Result {i+1} ') 
    eve_mesg , bob_mesg = main(n, sample_size)
    print('\n')
print(f'Interference detected {detected} times. It went undetected {undetected} times.')

Result 1 
Alice key  [0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0]
Bob key  [0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
Key matches?  False
Sample matches?  False
Eve's interference was detected.


Result 2 
Alice key  [1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0]
Bob key  [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0]
Key matches?  False
Sample matches?  False
Eve's interference was detected.


Result 3 
Alice key  [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]
Bob key  [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0